<a href="https://colab.research.google.com/github/4precious/ai/blob/main/GoldenChild_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-k1m1rtv4
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-k1m1rtv4
     |████████████████████████████████| 129 kB 5.1 MB/s 
     |████████████████████████████████| 54.7 MB 25 kB/s 
     |████████████████████████████████| 4.5 MB 51.4 MB/s 
     |████████████████████████████████| 1.2 MB 52.5 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:37tcmalloc: large alloc 1147494400 bytes == 0x39456000 @  0x7fa06625a615 0x58ead6 0x4f355e 0x4d222f 0x51041f 0x5b4ee6 0x58ff2e 0x510325 0x5b4ee6 0x58ff2e 0x50d482 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4d00fb 0x50cb8d 0x4bac0a 0x538a76 0x590ae5 0x510280 0x5b4ee6 0x58ff2e 0x50d482 0x5b4ee6 0x58ff2e 0x50c4fc 0x58fd37 0x50ca37 0x5b4ee6 0x58ff2e
     |████████████████████████████████| 881.9 MB 1

In [ ]:
import torch
from torch import nn
import torch.nn.functional as Functional
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, tqdm_notebook
import gluonnlp as nlp
import numpy as np

In [ ]:
# kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# using GPU
device = torch.device("cuda:0")

In [ ]:
import os

GPU_devices = torch.cuda.device_count()
print(GPU_devices)

for i in range(GPU_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
if torch.cuda.is_available():    
    GPU_device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('GPU that will be used:', torch.cuda.get_device_name(0))
else:
    GPU_device = torch.device("cpu")
    print('There is no GPU available.')

There are 1 GPU(s) available.
GPU that will be used: Tesla T4


In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir='.cache')

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Golden Child/data/감성대화말뭉치(최종데이터)_Training.xlsx')

In [ ]:
df

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40874,50789,청년,남성,대인관계,해당없음,불안,혼란스러운,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,친구 애완견을 대부분 혼자 돌보시는군요. 떠넘겨질까 봐 불안하시겠어요.,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.,친구분의 생각이 궁금해 얘기해보려고 하시는군요.,이대로 내가 키우게 되면 안 되니까 확실하게 해야겠어.,그렇게 하셨을 떄 지금의 상황이 어떻게 되기를 바라시나요?,친구가 자기가 데려온 강아지에 대해 책임감을 갖고 스스로 잘 돌봤으면 좋겠어.,친구에게 얘기해서 혼자 부담을 떠안지 않으면 좋겠어요.
40875,48959,중년,남성,건강,해당없음,기쁨,안도,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.,검진 결과가 좋게 나와서 안도하겠네요.,결과가 좋게 나와서 다행이야,건강검진에서 좋은결과를 받으신 상황이군요.,NaN,NaN,NaN,NaN
40876,36344,청소년,여성,가족관계,해당없음,슬픔,우울한,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,어머니의 응원이 사라져서 슬프시겠어요. 조금 더 자세히 말해주시겠어요?,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.,이 상황에서 무엇을 시도해 보고 싶나요?,내가 다시 내 꿈을 어떻게 이룰 것인지 자세히 설명해 드려야겠어.,꿈을 어떻게 이룰 것인지 엄마에게 이야기해보려 하시는군요.,NaN,NaN
40877,14197,노년,여성,재정,만성질환 무,기쁨,감사하는,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.,운동 시설을 쓸 수 있어서 기분이 좋으시겠어요.,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...,운동할 때 안마기를 사용하면 어떻게 도움이 되나요?,운동으로 뭉친 근육을 풀어주는 것 같아. 그럼 덜 피로하겠지.,안마기로 피로가 많이 풀리시길 바라요.,NaN,NaN


In [ ]:
df.columns

Index(['번호', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템응답1', '사람문장2', '시스템응답2', '사람문장3', '시스템응답3', '사람문장4', '시스템응답4'],
      dtype='object')

In [ ]:
# delete 'system response' columns that just replies to sentence
df = df[['감정_대분류', '감정_소분류', '사람문장1', '사람문장2']]
df

,감정_대분류,감정_소분류,사람문장1,사람문장2
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.
1,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.
2,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.
3,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...
4,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.
...,...,...,...,...
40874,불안,혼란스러운,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.
40875,기쁨,안도,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어.,결과가 좋게 나와서 다행이야
40876,슬픔,우울한,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.
40877,기쁨,감사하는,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야.,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...


In [ ]:
df.isnull().sum()

감정_대분류        0
감정_소분류        0
사람문장1         0
시스템응답1        0
사람문장2         0
시스템응답2        0
사람문장3      9820
시스템응답3     9820
사람문장4     39407
시스템응답4    39517
dtype: int64

In [ ]:
df.sample(10)

,감정_대분류,감정_소분류,사람문장1,사람문장2
12828,불안,걱정스러운,내 친구가 요즘 극단적인 생각을 하더라.,아무래도 애가 취업에 계속 실패하고 그런 증상이 잦아진 거 같아.
19414,슬픔,슬픔,아내가 암에 걸렸다는 걸 이제야 알게 되었어. 너무 슬퍼.,일부러 나에게 비밀로 하고 있었어. 내게 말을 안 했다는 것이 더 슬퍼.
11894,당황,외로운,나는 말을 잘 듣고 동생은 사고만 치는데 엄마는 나만 혼내. 너무 외롭고 속상해.,나는 작은 실수를 해도 혼나는데 동생은 봐주니까 서러워서 울고 싶어.
34484,상처,배신당한,나에게는 예쁜 여동생이 있어.,시집가지 않고 같이 평생 살자고 했는데 내일 모레 결혼을 한데 배신감이 들어.
39937,상처,충격 받은,평소에 아픈 곳이 없었는데 갑자기 몸이 안 좋아져서 병원에 다녀왔어.,병원에서 위가 좀 안 좋아졌다는 얘기를 듣고 충격받았어.
23283,슬픔,낙담한,장래 희망이 있는데 지금 내 성적으로는 꿈을 이루기에 절대 불가능할 것 같아.,그런 편이지. 지금 내 성적으로는 어림도 없어.
38141,기쁨,만족스러운,오랫동안 비염으로 고생했는데 친구가 소개해준 의사 덕분에 비염이 완치됐어.,정말 건강이 중요한 것 같아. 앞으로도 건강에 신경 써야겠어.
14939,슬픔,실망한,학년이 올라가면 자연스럽게 친구가 생길 거라도 생각했는데 나에게 관심을 가져주지 않...,따돌림을 당하고 있는 상황이 해결될 수 있을 거라 생각했는데 반이 바뀌었는데도 친구...
35335,불안,조심스러운,내가 피해를 준 친구 어머니를 만나 뵙고 사과 드리기로 했어.,처음에는 고민을 많이 했었는데 이게 맞는 것 같아 조심스럽게 이야기를 꺼냈어.
17667,기쁨,편안한,내일 딸이랑 영화 보러 가기로 했어. 언제 이렇게 컸는지 이젠 친구 같은 느낌이 들어.,내 인생 가장 큰 행복이지. 딸이 연애 고민을 상담해올 땐 내심 당황스럽더라고.


In [ ]:
df = df.melt(id_vars=['감정_대분류', '감정_소분류'], value_vars=['사람문장1', '사람문장2'])[['감정_대분류', '감정_소분류', 'value']]
df

,감정_대분류,감정_소분류,value
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.
1,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
3,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.
4,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.
...,...,...,...
81753,불안,혼란스러운,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.
81754,기쁨,안도,결과가 좋게 나와서 다행이야
81755,슬픔,우울한,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.
81756,기쁨,감사하는,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...


In [ ]:
df.rename(columns = {'value':'Sentence'},inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.감정_대분류.unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [ ]:
df['감정_대분류'] = df['감정_대분류'].str.replace(' ','')

In [ ]:
df.감정_대분류.unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처'], dtype=object)

In [ ]:
dict1 = {'기쁨': 0, '불안': 1, '당황': 2, '슬픔': 3, '분노': 4, '상처': 5}
dict1

{'기쁨': 0, '불안': 1, '당황': 2, '슬픔': 3, '분노': 4, '상처': 5}

In [ ]:
dict1['기쁨']

0

In [ ]:
df['감정_대분류_int'] = -1
df

,감정_대분류,감정_소분류,Sentence,감정_대분류_int
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,-1
1,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,-1
2,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,-1
3,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,-1
4,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,-1
...,...,...,...,...
81753,불안,혼란스러운,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.,-1
81754,기쁨,안도,결과가 좋게 나와서 다행이야,-1
81755,슬픔,우울한,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.,-1
81756,기쁨,감사하는,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...,-1


In [ ]:
df['감정_대분류']

0        기쁨
1        불안
2        당황
3        기쁨
4        기쁨
         ..
81753    불안
81754    기쁨
81755    슬픔
81756    기쁨
81757    불안
Name: 감정_대분류, Length: 81758, dtype: object

In [ ]:
[df['감정_대분류'].values]

array(['기쁨', '불안', '당황', ..., '슬픔', '기쁨', '불안'], dtype=object)

In [ ]:
df['감정_대분류'].map(dict1)

0        0
1        1
2        2
3        0
4        0
        ..
81753    1
81754    0
81755    3
81756    0
81757    1
Name: 감정_대분류, Length: 81758, dtype: int64

In [ ]:
df['감정_대분류_int'] = df['감정_대분류'].map(dict1)
df

,감정_대분류,감정_소분류,Sentence,감정_대분류_int
0,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,1
2,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,2
3,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
...,...,...,...,...
81753,불안,혼란스러운,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.,1
81754,기쁨,안도,결과가 좋게 나와서 다행이야,0
81755,슬픔,우울한,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.,3
81756,기쁨,감사하는,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...,0


In [ ]:
dataset = df[['Sentence', '감정_대분류_int']]
dataset

,Sentence,감정_대분류_int
0,아내가 드디어 출산하게 되어서 정말 신이 나.,0
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,2
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,0
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,0
...,...,...
81753,나한테 아예 떠넘길 거 같은데 친구한테 얘기해볼까 고민 중이야.,1
81754,결과가 좋게 나와서 다행이야,0
81755,내 재능이 남들보다 월등한 거는 아니라면서 취업해서 안정적으로 살았으면 좋겠다고 하셔.,3
81756,덕분에 건강도 챙길 수 있고 너무 좋아. 안마기도 있던데 내일은 운동하고 안마기도 ...,0


In [ ]:
dataset_list = dataset.values.tolist()

In [ ]:
dataset_train, dataset_test = train_test_split(dataset_list, test_size=0.25, random_state=0)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))  

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
data_train[0]

(array([   2, 3647, 1397, 5330,  517, 5890, 1493, 7289, 6116, 4136, 4998,
        5737, 7318, 2086, 5007, 5176, 7953, 5876,  517,   54,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(21, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 1/959 [00:00<14:42,  1.09it/s]

epoch 1 batch id 1 loss 1.8428150415420532 train acc 0.21875


 11%|█         | 101/959 [01:12<09:54,  1.44it/s]

epoch 1 batch id 101 loss 1.8022171258926392 train acc 0.16785272277227722


 21%|██        | 201/959 [02:21<08:51,  1.43it/s]

...

epoch 15 batch id 901 loss 0.06113147735595703 train acc 0.9644318812430632


100%|██████████| 959/959 [11:11<00:00,  1.43it/s]

epoch 15 train acc 0.9643671793534933



100%|██████████| 320/320 [01:17<00:00,  4.10it/s]

epoch 15 test acc 0.5775390625


In [ ]:
## save the trained model
PATH = '/content/drive/MyDrive/Golden Child/model/'
torch.save(model, PATH + 'KoBERT_model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')

In [2]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("불안이")
            elif np.argmax(logits) == 2:
                test_eval.append("당황이")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("분노가")
            elif np.argmax(logits) == 5:
                test_eval.append("상처가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

NameError: ignored

In [1]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 룰룰


NameError: ignored